**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys

# install deps for explain manually, since pdpbox requires
# an ancient version of matplotlib as a dep
!{sys.executable} -m pip install --no-deps pdpbox lime eli5
!{sys.executable} -m pip install class_utils@git+https://github.com/michalgregor/class_utils.git
#!{sys.executable} -m pip install class_utils[explain]@git+https://github.com/michalgregor/class_utils.git

!{sys.executable} -m pip install xgboost

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from class_utils import Explainer
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

try:
    import google.colab
    COLAB_MODE = True
except:
    COLAB_MODE = False

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Interpreting Models

There is a lot of applications, where it is not enough to train a model and compute predictions. We need to be able to interpret the model and to explain why a certain prediction was made. Such interpretability is a vital prerequisite of safe and trustworthy machine learning and artificial intelligence – it helps to verify that a system is not biased and that its predictions are not based on protected attributes such as race. In some countries, interpretability is even required by law: In the EU, for instance, whenever an automatic system makes decisions about humans they have the right to an explanation.

There are models, which have some inherent interpretability. Decision trees are a good example: a tree is essentially just a collection of rules. We can plot it and read through it. With most kinds of models, however, this is not possible. Even with decision trees it becomes harder with increasing size and as soon as we form an ensemble of decision trees, it is simply not practicable anymore.

In this notebook we will showcase a few generic methods that help to interpret predictions of arbitrary models. We will again start by loading and preprocessing the [Titanic](https://www.kaggle.com/c/titanic) dataset.



In [ ]:
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']
output = "Survived"

class_names = [ "died", "survived"]

In [ ]:
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values.reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values.reshape(-1)

We will extract the imputers from our pipeline. These are important: they will be used later when constructing an explainer.



In [ ]:
categorical_imputer = input_preproc.transformers_[0][1][0]
numeric_imputer = input_preproc.transformers_[1][1][0]

We will next train an XGBoost model on the data and compute its accuracy on the test set just to make sure that everything works correctly.



In [ ]:
model = XGBClassifier()
model.fit(X_train, Y_train)

In [ ]:
y_test = model.predict(X_test)
accuracy_score(Y_test, y_test)

We create an explainer: an auxiliary object that will allow us to create the explanations. When constructing the explainer, it is crucial to use **the same kind of imputation of missing values**  that we used to train our model. Otherwise we will be explaining different samples than the model would normally see.



In [ ]:
explainer = Explainer(
    model, df_train,
    categorical_inputs,
    categorical_imputer,
    numeric_inputs,
    numeric_imputer,
    input_preproc,
    class_names
)

### Permutation Importance

The first thing that it might be useful to know is the relative influence of individual features on the prediction (feature importance). One well-known way to compute the importance of a feature is to permute its column in the dataset (to shuffle it so that it gets out of order) and observe how that affects the predictions. If they change very severely the feature was presumably very important to the prediction. If they only change slightly or not at all, the importance of the feature is probably negligible. 



In [ ]:
perm = explainer.permutation_importance(df_test, Y_test)

In Titanic, for instance, column "Sex" seems to be the most important by far. This indicates that men and women have very different rates of survival.

### Partial Dependence Plots

To investigate the influence of a feature on the prediction in more detail, we can use the so called partial dependence plots. These are formed by systematically changing a single feature and observing how that influences the results. Let us see what the partial dependence on column "Sex" then. Recall that we are predicting whether a person survived (1) or not (0). A positive number means that a feature contributes to survival and a negative number indicates that it contributes to death.



In [ ]:
explainer.pdp_plot(df_test, "Sex")

It is obvious that being male can significantly decrease the chance of survival as far as our classifier is concerned. However, there is a lot more variance than for women – this means that at least for some men there would still be at least a reasonable chance of survival.

If we explore the PD plot for "Fare", we should see a positive relationship: higher fares generally meant a better chance of survival.



In [ ]:
explainer.pdp_plot(df_test, "Fare")

The plots do not have to be monotonous. For age, for example, the situation is a bit more nuanced – although there is a chance that this is due to noise in the data: the effect is not that pronounced.



In [ ]:
explainer.pdp_plot(df_test, "Age")

### LIME: Local Interpretations

Finally, we might be interested in local interpretations: when given a particular sample, we might want to know the influence that each feature had on the prediction. There is a method called LIME (Local Interpretable Model-agnostic Explanations), which provides this kind of explanation by fitting a local linear model around the prediction. This makes the approach model-agnostic: it works with any kind of model.

To experiment with LIME, we will pick some sample from the dataset and have it explained. We will see which features have a positive and a negative influence on a particular prediction and what the magnitude of that influence is.



In [ ]:
exp = explainer.explain(df_test.iloc[2])
exp.show_in_notebook(show_all=True, colab_mode=COLAB_MODE)

In [ ]:
exp.as_pyplot_figure()
plt.show()